In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
import os
from PIL import Image
from sklearn.preprocessing import LabelEncoder  # label 수로 변환
import cv2

## 참고 자료
https://www.datacamp.com/tutorial/pytorch-tutorial-building-a-simple-neural-network-from-scratch
https://gaussian37.github.io/dl-pytorch-neural-network/
https://learn.microsoft.com/ko-kr/windows/ai/windows-ml/tutorials/pytorch-train-model


In [2]:
# 데이터 폴더 경로
path = 'C:\\Users\\JungHyeona\\Desktop\\KNN\\image'

In [3]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img = self.images[index]
        label = self.labels[index]
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, label

In [4]:
images = []
labels = []

for for_name in os.listdir(path):  # i = ["AC", "FL", "HC", "HUM"]
    forder = os.path.join(path, for_name)  # ex) path/AC

    for j in os.listdir(forder):
        file = os.path.join(forder, j)  # ex) path/AC/a.png
        
        image = Image.open(file)
        label = os.path.basename(os.path.dirname(file))
        
        images.append(image)
        labels.append(label)

In [5]:
# label값을 숫자로 변환해주기
label_trans = {"AC" : 0, "FL" : 1, "HC" : 2, "HUM" : 3}
labels = [label_trans[label] for label in labels]

### 데이터 전처리

In [6]:
transform = transforms.Compose([
     transforms.Resize((32, 32)),
     transforms.ToTensor(),
     transforms.Normalize((-0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [7]:
# 데이터셋 인스턴스 생성
dataset = CustomDataset(images, labels, transform=transform)

In [8]:
# 데이터셋을 train, validation, test로 분리
train_dataset, test_dataset = train_test_split(dataset, test_size=0.3, random_state=42)

In [9]:
# DataLoader를 통해 데이터를 불러올 수 있도록 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [19]:
len(train_loader)

27

In [11]:
len(test_dataset)

365

In [20]:
# 이미지 크기 확인
for batch, (X, y) in enumerate(train_loader):
    print(batch)
    print(X.shape)
    print(y.shape)
    break

0
torch.Size([32, 3, 32, 32])
torch.Size([32])


In [22]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

# Create a datas et with 10,000 samples.
X, y = make_circles(n_samples = 10000,
                    noise= 0.05,
                    random_state=26)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=26)

In [27]:
X_train

array([[-0.86143957,  0.51365619],
       [ 0.15802293,  0.75413978],
       [ 0.10045332, -1.02042385],
       ...,
       [-0.03391136, -0.79715534],
       [-0.56320429,  0.88155148],
       [ 0.94764034,  0.31432114]])

In [38]:
print([train_dataset[-i][1] for i in range(10)])

[2, 3, 2, 3, 3, 3, 0, 1, 1, 0]


In [ ]:
print(np.unique(train

### Neural Network 모델

In [14]:
class NN(nn.Module):

    # 신경망 layer 구성요소 정의
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linear01 = nn.Linear(input_dim, hidden_dim)
        self.linear02 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self,x):
        x = F.relu(self.linear01(x))
        x = F.relu(self.linear02(x))

        return x
        
model = NN(32*32, 18, 4)
# 학습하기 전의 weight와 bias를 출력합니다.
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.5071, -0.4946,  0.0399],
        [-0.5402, -0.2331, -0.3610],
        [ 0.3152,  0.5013,  0.1851],
        [-0.0150, -0.2898,  0.4964],
        [-0.3974, -0.0683, -0.5710],
        [-0.5149, -0.3681, -0.4311],
        [ 0.0793,  0.5364, -0.0754],
        [ 0.2094,  0.4978,  0.2162],
        [-0.0772,  0.2597, -0.0636],
        [-0.0963, -0.5336, -0.3171],
        [-0.3007,  0.4303, -0.5254],
        [ 0.1989, -0.0021, -0.2080],
        [ 0.2693, -0.3845, -0.2466],
        [-0.3725,  0.0975, -0.5127],
        [-0.3596, -0.1969, -0.2168],
        [-0.1959, -0.1764, -0.2431],
        [-0.5233, -0.0869,  0.3048],
        [ 0.2132, -0.3506, -0.1781]], requires_grad=True), Parameter containing:
tensor([-0.0462,  0.3969, -0.0921,  0.1233,  0.1943, -0.4459,  0.4148,  0.0579,
        -0.5382, -0.1294, -0.2612, -0.1981, -0.2078,  0.0231,  0.3558, -0.3537,
         0.4587,  0.5065], requires_grad=True), Parameter containing:
tensor([[-0.2070,  0.2053, -0.2113, -

In [15]:
# GPU 사용 가능 여부에 따라 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model.to(device)

Using cuda device


NN(
  (linear01): Linear(in_features=3, out_features=18, bias=True)
  (linear02): Linear(in_features=18, out_features=4, bias=True)
)

### 손실함수

In [16]:
loss_func = nn.CrossEntropyLoss()
learning_rate = 0.0002
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

In [17]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 27


In [18]:
loss_values = []
avg_cost = 0

for epoch in range(20):
    
    # 미니 배치 단위로 꺼내온다.
    for X, Y in train_loader: 
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)
        
        # 파라미터(가중치) 0으로 초기화
        optimizer.zero_grad()
        
        # 예측과 손실 계산
        m_pred = model(X)
        loss = loss_func(m_pred, Y)
        loss_values.append(loss.item())
        
        # 역전파
        loss.backward()
        
        # 경사 갱신
        optimizer.step()
        
        avg_cost += loss / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (3072x32 and 3x18)

### 모델 평가 함수

In [22]:
def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [ ]:
# 모델 평가
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
val_accuracy = evaluate(model, val_loader, device)
print(f'Validation Accuracy: {val_accuracy:.2f}%')

In [23]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [24]:
x_train

tensor([[ 73.,  80.,  75.],
        [ 93.,  88.,  93.],
        [ 89.,  91.,  90.],
        [ 96.,  98., 100.],
        [ 73.,  66.,  70.]])